### Real, Complex and Symplectic Reflection Groups - March 2023, RUB

## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 2. Finite Coxeter Groups in Action

[![Vitruvian Man](images/vitruvian.jpg)](https://en.wikipedia.org/wiki/Vitruvian_Man)

## Setup

We use the graph traversal techniques from the last day to
* construct a finite Coxeter group as a permutation group,
* compute basic properties of elements and parabolic subgroups,
* determine the normalizer complement of a parabolic subgroup,
* examine the conjugation action with respect to element length.

First, reload the algorithms ...

In [ ]:
Read("gap4/orbits.g");
LoadPackage("jupyterviz");
opts:= rec(vertexwidth:= 12, vertexheight:= 12, edgecolor:= "#def");;

## Real Reflection Groups

<div class="alert alert-danger">

**Definition.**

* Let $S$ be a finite set, and let $M = (m_{st})$ be a symmetric matrix with $m_{ss} = 1$
and $m_{st} = m_{ts} \in \{2,3,4,\dots,\infty\}$ if $s \neq t$.
* A **Coxeter group** with  **Coxeter Matrix** $M$ is the group $W$ given by the presentation
    $$
    W = \langle S \mid (st)^{m_{st}} = 1;\, s,t \in S \rangle
    $$
</div>

* The presentation of a Coxeter group can be described by a graph with vertex set $S$ and edges labelled $m_{st}$  between $s$ and $t$ if $m_{st} > 2$, where for $m_{st} = 3$ the label is usually omitted.
* A Coxeter group is called **simply laced** if $m_{st} \leq 3$ for all $s, t \in S$.
* Coxeter graphs (for simply laced irreducible finite Coxeter groups) can be made by the following GAP program.

In [ ]:
coxeterGraph := function(series, rank)
    local edges;
    edges := List([2..rank], j -> [j-1, j]);
    if series >= "D" then  edges[1][2] := 3;  fi;
    if series >= "E" then  edges[2][2] := 4;  fi;
    return edges;
end;

* For example:

In [ ]:
graph := coxeterGraph("A", 7);

In [ ]:
PlotGraph(graph, opts);

* From the Coxeter matrix $M$, or the corresponding graph, we set up a Cartan matrix $C$:
* $C = (c_{st})$ with $c_{ss} = 2$ and $c_{st} c_{ts} = 4 \cos^2 \frac{\pi}{m_{st}}$, $c_{st} = 0 \iff c_{ts} = 0$, $s \neq t$.

In [ ]:
cartanMat := function(series, rank)
    local cartan,  ij,  i,  j;
    cartan := 2*IdentityMat(rank);;
    for ij in coxeterGraph(series, rank) do
        i := ij[1];  j := ij[2];
        cartan[i][j] := -1;
        cartan[j][i] := -1;
    od;
    if series = "B" then  cartan[1][2] := -2;  fi;
    if series = "C" then  cartan[2][1] := -2;  fi;
    if series = "F" then  cartan[3][4] := -2;  fi;  # sic!
    return cartan;
end;

In [ ]:
C := cartanMat("A", 3);

In [ ]:
PrintArray(C);

* From the Cartan matrix $C$ we can compute matrices for the **simple reflections**: the space $V = \mathbb{R}^n$ has a basis of **simple roots** $\{\alpha_s : s \in S\}$ and
$$
\alpha_t.\tilde{s} = \alpha_t - c_{st} \alpha_s
$$
defines a linear action $\tilde{s} \in \mathrm{GL}(V)$ of $s$ as **reflection** in the hyperplane orthogonal to $\alpha_s$.

In [ ]:
S := [1..Length(C)];;  mats := [];;  one := C^0;;
for s in S do
    mats[s] := C^0;  mats[s]{S}[s] := one[s] - C[s];
od;

In [ ]:
for mat in mats do
    PrintArray(mat);  Print("\n");
od;

In [ ]:
mats[1]^2;

* Thus, $W = \langle \tilde{s} : s \in S \rangle \leq \mathrm{GL}(V)$ acts as a **reflection group** on $V$.
* Note that $\{\alpha_s : s \in S\}$ is not the standard basis of $V$.
* But there are real numbers $d_s$, $s \in S$ such that
$$
(\alpha_s, \alpha_t) = \tfrac12 d_s^2 c_{st}
$$
defines a $W$-invariant bilinear form on $V$ (with $d_s = \sqrt{(\alpha_s, \alpha_s)}$).

* The **root system** $\Phi$ of $W$ (or of $C$) is $\Phi = \{\alpha_s.w : s \in S,\,w \in W\}$, a union of $W$-orbits.
* So we can grow this root system as **orbits** of the simple roots $\alpha_s$ (which, as our basis vectors,  are simply the rows of the identity matrix $C^0$).

In [ ]:
phi := orbits(mats, C^0, OnRight);

* Note how $\Phi = \Phi^{+} \cup \Phi^{-}$, where $\Phi^{+} = \{ \alpha \in \Phi : \alpha \geq 0\}$.
* Or, how roots come in pairs of negatives $\{r, -r\}$.  We could enumerate them as such ...

In [ ]:
onRootPairs := function(pair, a)
    return Set(OnRight(pair, a));
end;;
phi := orbits(mats, List(C^0, x -> [-x, x]), onRootPairs);

* ... and obtain $\Phi$ by rearranging the pairs.

In [ ]:
phi := Concatenation(TransposedMat(phi){[2,1]});

* Or, we only enumerate the positive representatives of each pair ...

In [ ]:
absRoot := root -> SignInt(Sum(root)) * root;;
onRoots := function(x, a)
    return absRoot(OnRight(x, a));
end;;
phi:= orbits(mats, C^0, onRoots);

* ... and find $\Phi$ as concatentation of $\Phi^{+}$ and $- \Phi^{+}$.

In [ ]:
phi:= Concatenation(phi, -phi);

* In fact, for future reference, we can compute the edges of the action graph, and words in the generators along with $\Phi$. (What are suitable words for the initial roots?  What are the initial edges?)

In [ ]:
orbits_with_words_and_edges := function(aaa, xxx, under)
    local   list,  edges,  words,  i,  k,  l,  z;
    words := List([1..Length(xxx)], i -> [i]);
    list := ShallowCopy(xxx);  edges := [];  i := 0;
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            l := Position(list, z);
            if l = fail then
                Add(list, z);
                Add(words, onWords(words[i], k));
                l := Length(list);
            fi;
            Add(edges, [i, l]);
        od;
    od;
    return rec(list := list, edges := edges, words := words);
end;

In [ ]:
roots := orbits_with_words_and_edges(mats, C^0, onRoots);;
roots.list;

In [ ]:
edges := Filtered(roots.edges, x-> x[1] <> x[2]);
PlotGraph(edges, opts);

In [ ]:
roots.words;

* Express simple reflections as permutations of the roots. Formula?  (Compare with Schreier generators $(f_y a)/f_{y.a}$!)
* `Sortex` returns the permutation needed to sort a list: `Permuted(list, Sortex(list))` is the sorted list.
* So, if $a$ maps `xxx` to `yyy`, $b$ sorts `yyy` (i.e., maps it to `zzz`) and $c$ sorts `xxx` (i.e., maps it to `zzz`) then $c/b$ maps `xxx` to `yyy`.
* The permutation of `xxx` caused is the map that assigns $i \mapsto j$ if `yyy[i] = xxx[j]`, i.e. $b/c = (c/b)^{-1}$ (Check!).

In [ ]:
permutation := function(a, xxx, under)
    return Sortex(List(xxx, x-> under(x, a))) / Sortex(ShallowCopy(xxx));
end;

In [ ]:
permutation(mats[1], phi, OnRight);

In [ ]:
permutation(mats[1], phi, OnRight) * permutation(mats[2], phi, OnRight);

In [ ]:
permutation(mats[1] * mats[2], phi, OnRight);

In [ ]:
a := mats[1] * mats[2];

In [ ]:
Permuted(phi, permutation(a, phi, OnRight)^-1) = List(phi, x-> x * a);

* GAP has a function `Permutation` for this purpose.

In [ ]:
Permutation(a, phi, OnRight);

In [ ]:
perms:= List(mats, m-> Permutation(m, phi, OnRight));

In [ ]:
perms[1] * perms[2];

* all in one program

In [ ]:
DeclareAttribute("Data", IsPermGroup);

coxeterGroup := function(C)
    local  one,  mats,  roots,  S,  s,  phi,  data,  G;
    one := C^0;  mats := [];  S := [1..Length(C)];
    for s in S do
        mats[s] := C^0;  mats[s]{S}[s] := one[s] - C[s];
    od;
    roots := orbits_with_words_and_edges(mats, C^0, onRoots);
    data := rec(mats := mats, roots := roots, rank := Length(S));
    data.N := Length(roots.list);
    data.phi := Concatenation(roots.list, -roots.list);
    data.perms := List(mats, m -> Permutation(m, data.phi, OnRight));
    G:= GroupWithGenerators(data.perms);
    SetData(G, data);
    return G;
end;

In [ ]:
W:= coxeterGroup(C);

In [ ]:
Data(W);

In [ ]:
sizeOfGroup(W);

* E8 ...

In [ ]:
graphE8:= coxeterGraph("E", 8);

In [ ]:
PlotGraph(graphE8, opts);

In [ ]:
cartanE8:= cartanMat("E", 8);

In [ ]:
E8:= coxeterGroup(cartanE8);;

In [ ]:
E8.1;

In [ ]:
sizeOfGroup(E8);

In [ ]:
Data(E8).N;

In [ ]:
edges:= Filtered(Data(E8).roots.edges, x-> x[1] <> x[2]);;
PlotGraph(edges);

### Reflections

In [ ]:
reflections := function(W)
    local reflection_from_word;
    reflection_from_word := function(w)
        return W.(w[1])^Product(Data(W).perms{w{[2..Length(w)]}});
    end;
    return List(Data(W).roots.words, reflection_from_word);
end;

In [ ]:
A3 := coxeterGroup(cartanMat("A", 3));
rr := reflections(A3);

In [ ]:
Length(rr);

##  Properties

* Some Basis properties are now immediate.

### Length

* the **length** of  an element $w \in W$ is $\ell(w) = \#\{\alpha \in \Phi^{+} : \alpha.w \in \Phi^{-}\}$

In [ ]:
coxeterLength:= function(W, w)
    return Number([1..Data(W).N], i-> i^w > Data(W).N);
end;

In [ ]:
List(Data(E8).perms, w-> coxeterLength(E8, w));

In [ ]:
word:= [5,2,3,8,7,5,3,1,6];
perm:= Product(Data(E8).perms{word});
coxeterLength(E8, perm);

In [ ]:
permCoxeterWord:= function(W, word)
    if word = [] then return (); fi;
    return Product(Data(W).perms{word});
end;

In [ ]:
permCoxeterWord(E8, word);

### Descents

* $s \in S$ is a **left descent** of $w \in W$ if $\ell(sw) < \ell(w)$, i.e., if $\alpha_s.w \in \Phi^{-}$.

In [ ]:
isLeftDescent:= function(W, w, s)
    return s^w > Data(W).N;
end;

In [ ]:
isLeftDescent(E8, perm, 5);
isLeftDescent(E8, perm, 2);

### Reduced Expressions

* perm -> word:  $w$ as a word in $S$ is a sequence of left descents.

In [ ]:
coxeterWord:= function(W, w)
    local word, a;
    word:= [];
    while w <> () do
        a:= First([1..Data(W).rank], s-> isLeftDescent(W, w, s));
        Add(word, a); w:= Data(W).perms[a] * w;
    od;
    return word;
end;

In [ ]:
reduced:= coxeterWord(E8, perm);

In [ ]:
permCoxeterWord(E8, reduced) = perm;

* to find a reduced expression of any word in $S$, convert word -> perm and perm -> word.

In [ ]:
reducedWord:= function(W, word)
    return coxeterWord(W, permCoxeterWord(W, word));
end;

In [ ]:
word;
reducedWord(E8, word);

### Longest Elements

* Longest elements.  For each $J \subseteq S$ there is one, $w_J$, a product of non-descents. 

In [ ]:
longestElement:= function(W, J)
    local  wJ,  s;
    wJ:= ();
    while true do
        s:= First(J, s-> not isLeftDescent(W, wJ, s));
        if s = fail then  return wJ;  fi;
        wJ:= W.(s) * wJ;
    od;
end;

In [ ]:
J:= [4,5,6];
wJ:= longestElement(E8, J);
coxeterWord(E8, wJ);

* $w_J$ acts as a graph automorphism on $W_J$.

In [ ]:
w6:= longestElement(E8, [1..6]);
Permutation(w6, Data(E8).perms{[1..6]}, OnPoints);

###  Prefixes

* Prefixes (aka weak Bruhat Order): $u \in W$ is a **prefix** of $w \in W$ if $\ell(u) + \ell(u^{-1} w) = \ell(w)$, i.e., if $w = uv$ with $\ell(w) = \ell(u) = \ell(v)$.
The set of all prefixes of $w$ is the **orbit (!)** of $w$ under the action $(w, s) \mapsto ws$ if $\ell(ws) < \ell(w)$, and $w$ else.

In [ ]:
prefixes:= function(W, w)
    local onRightDescents;

    onRightDescents:= function(w, s)
        if isLeftDescent(W, w^-1, s) then
            return w * W.(s);
        else
            return w;
        fi;
    end;

    return orbit([1..Data(W).rank], w, onRightDescents);
end;

In [ ]:
w:= longestElement(E8, [4,5]);;
pre:= prefixes(E8, w);;
List(pre, x-> CoxeterWord(E8, x));

### Distinguished Coset Representatives

* A coset $W_J w$ of a parabolic subgroup $W_J$ of $W$ contains a **unique element of minimal length**.
* The parabolic subgroup thus has a **distinguished transversal** $X_J = \{ x \in W : \ell(ux) = \ell(u) + \ell(x) \text{ for all } u \in W_J \}$.
* $X_J$ is the set of prefixes of the **longest coset representative** $d_J = w_J w_S$.

In [ ]:
prefixes_with_edges:= function(W, w)
    local onRightDescents;

    onRightDescents:= function(w, s)
        if isLeftDescent(W, w^-1, s) then
            return w * W.(s);
        else
            return w;
        fi;
    end;

    return orbit_with_edges([1..Data(W).rank], w, onRightDescents);
end;

In [ ]:
J:= [1..6]; K := [1..7];
wJ:= longestElement(E8, J);;
wK:= longestElement(E8, K);;
dJ:= wJ * wK;
XJ:= prefixes_with_edges(E8, dJ);;

In [ ]:
edges := Filtered(XJ.edges, x-> x[1] <> x[2]);;
PlotGraph(edges, opts);

* Let's wrap $d_J^L = w_J w_L$ into a short function.

In [ ]:
longestCosetElement:= function(W, J, L)
    return longestElement(W, J) * longestElement(W, L);
end;

In [ ]:
coxeterWord(E8, longestCosetElement(E8, J, K));

##  Shapes: Classes of Parabolic Subgroups

* $W$ acts on its subgroups by conjugations, in particular on its parabolic subgroups.
* The set of standard parabolic subgroups $W_J$, $J \subseteq S$ is thus partitioned into **shapes**: classes of conjugate parabolics.

* Obviously, if $L = J \cup \{s\}$ for some $s \in S \setminus J$ then conjugation by $d_J^L$ maps $W_J$ to
$W_K$ for some $K \subseteq L$.  In fact, $K = J^{d_J^L}$.

* **Theorem** (Lusztig-Spaltenstein, Howlett, Deodhar) The shape of $W_J$ is the $S^*$-orbit of $W_J$ under the above action.

In [ ]:
tackOn := function(x, s)
    return Union(x, [s]);
end;

In [ ]:
tackOn([1,3,5], 4);

In [ ]:
shape := function(W, J)
    local onParabolics;
    onParabolics := function(K, s)
        return OnSets(K, longestCosetElement(W, K, tackOn(K, s)));
    end;
    return orbit([1..Data(W).rank], J, onParabolics);
end;

In [ ]:
shape(E8, [1,3]);

 * Denote by $\Lambda$ the set of all shapes of $W$.
 * Recall how the power set $2^S$ is the orbit of $S^*$ under the `takeAway` action.
 * $\Lambda$ is the $S^*$-orbit of the shape $\{S\}$ under that same action.

In [ ]:
takeAway:= function(x, s)
    return Difference(x, [s]);
end;

In [ ]:
shapes := function(W)
    local onShapes, S;
    onShapes := function(x, s)
        return Set(shape(W, takeAway(x[1], s)));
    end;
    S := [1..Data(W).rank];
    return orbit(S, shape(W, S), onShapes);
end;

In [ ]:
sss := shapes(E8);

In [ ]:
Sum(sss, Length);

* A shape has a graph.

In [ ]:
shape_with_edges := function(W, J)
    local onParabolics;
    onParabolics := function(K, s)
        return OnSets(K, longestCosetElement(W, K, tackOn(K, s)));
    end;
    return orbit_with_edges([1..Data(W).rank], J, onParabolics);
end;

In [ ]:
sh := shape_with_edges(E8, [1,3,4]);

In [ ]:
edges := Filtered(sh.edges, x-> x[1] <> x[2]);;
PlotGraph(edges);

* In order to compute a transversal for the shape, we need to modify the algorithm.
* We need to adjust for the fact that a generator $s \in S \subseteq \mathbb{N}$ merely represents a conjugating element $d_K d_L \in W$, which even depends on the context.
* We might as well also take into account, that for the current parabolic $K$ only $s \notin K$ needs to be considered.

In [ ]:
shape_with_transversal := function(W, J)
    local   S,  list,  reps,  i,  K,  s,  a,  L;
    S := [1..Data(W).rank];  list:= [J];  reps:= [()];  i:= 0;
    while i < Length(list) do
        i := i+1;  K := list[i];
        for s in Difference(S, K) do
            a := longestCosetElement(W, K, tackOn(K, s));
            L := OnSets(K, a);
            if not L in list then
                Add(list, L);
                Add(reps, reps[i] * a);
            fi;
        od;
    od;
    return rec(list:= list, reps:= reps);
end;

##  Normalizers of Parabolic Subgroups

* **Theorem** (Howlett's Lemma)  Let $W$ be a reflection group on $V = \mathbb{R}^n$ with $W ⊴ G ≤ O(V)$. Then $W$ has a complement $H$ in $G$
* In particular, $N_W(W_J) = W_J : N_J$.
* In fact, $N_J$ is the **stabilizer** of $J$ in the action of $S^*$ on parabolics.

In [ ]:
parabolicComplement:= function(W, J)
    local   S,  list,  reps,  i,  gens,  K,  s,  a,  L,  j;
    S := [1..Data(W).rank];  list := [J];  reps := [()];
    gens := rec(ears := [], eyes := []);
    i := 0;
    while i < Length(list) do
        i := i+1;  K := list[i];
        for s in Difference(S, K) do
            a := longestCosetElement(W, K, tackOn(K, s));
            L := OnSets(K, a);  j := Position(list, L);
            if j = fail then
                Add(list, L);
                Add(reps, reps[i] * a);
            elif j = i then
                AddSet(gens.ears, reps[i] * a * reps[i]^-1);
            else
                AddSet(gens.eyes, reps[i] * a * reps[j]^-1);
            fi;
        od;
    od;
    return gens;
end;


In [ ]:
parabolicComplement(E8, [1,3,4]);

...

##  Conjugacy Class by Length

## Cyclic Shifts and Minimal Length Representatives

In [ ]:
byCyclicShift:= function(w, s)
    local y;
    y:= w^s;
    if coxeterLength(W, y) = coxeterLength(W, w) then
        return y;
    fi;
    return w;
end;

##  Involutions?

In [ ]:
E7 := coxeterGroup(cartanMat("E", 7));;

In [ ]:
cc := conjugacyClasses(E7);;
Length(cc);
Sum(cc, Size);

In [ ]:
onInvolutions := function(x, s)
    if x * s = s * x then return OnRight(x, s); fi;
    return OnPoints(x, s);
end;

In [ ]:
invos := orbit(Data(E7).perms, (), onInvolutions);;
Length(invos);

In [ ]:
onInvolutionClasses:= function(x, a)
    local y;
    y:= Representative(x);
    if y^a <> y then return x; fi;
    return OnRight(y, a)^ActingDomain(x);
end;

In [ ]:
involutionClasses := function(group)
    local gens;
    gens:= GeneratorsOfGroup(group);
    return orbit(orbits(gens, gens, OnPoints), Identity(group)^group, onInvolutionClasses);
end;

In [ ]:
invos := involutionClasses(E7);;
Length(invos);
Sum(invos, Size);

## Exercises, etc.

* Prove that $\Lambda$ is an orbit under `takeAway`.
* Prove that the complement $N_J$ is a stabiliser under `onParabolics`.
* Find an efficient way to enumerate **double cosets** $X_{JK} = X_J \cap X_K^{-1}$, $J, K \subseteq S$.